# Desafio Kinea


## Importando e definindo constantes globais

In [1]:
import requests
import base64 as b64
import json
import pandas as pd
from typing import Dict, List
import csv
from dateutil.relativedelta import relativedelta

In [68]:
from ambima_connect import AmbimaConnect

In [2]:
import logging
logging.level=logging.INFO

In [3]:
from datetime import date, datetime
from cal import AmbimaCalendar

## Funções básicas

In [5]:
def dias_uteis(start_date, end_date) -> int:
  """O formato de data é date(AAAA, MM, DD), como date(2022, 2, 4) para 4 de fev de 2022"""
  interval = AmbimaCalendar().get_working_days_delta(start_date, end_date)+1
  return interval

In [7]:
def check_error(acquired_value, expected_value):
  err = abs((acquired_value-expected_value)/expected_value)*100
  err_str = f'{err:0.3}%'
  return err_str

## Interface com API Ambima

In [ ]:
# data_res = get_data(client_id, access_token, adress_vna)
data_list = AmbimaConnect('titulos publicos').content
data_vna = AmbimaConnect('vna').content

In [14]:
def save_to_csv(filedata:List[Dict], filename:str):
  """Take a list of dicts and save as an .csv with the given name"""
  keys = filedata[0].keys()
  with open('exports\\'+filename, 'w', newline='') as output_file:
      dict_writer = csv.DictWriter(output_file, keys)
      dict_writer.writeheader()
      dict_writer.writerows(filedata)

In [67]:
def clean_mercado_secundario(list_to_clean:List[Dict]):
  keep_keys = ['tipo_titulo','data_referencia','data_vencimento','taxa_indicativa','pu']
  res = [{key : val for key, val in sub.items() if key in keep_keys} for sub in list_to_clean]
  return res

In [16]:
clean_mercado = clean_mercado_secundario(data_list)

## Cálculo de PUs

In [17]:
def get_dias_cupom(start_date, end_date):
  """Retorna uma listagem de cupons entre duas datas, indo da data final até a inicial, de 6 em 6 meses"""
  pgto_date_list = list()
  pgto_date = end_date
  # Gera da lista de datas a partir da data final, em incrementos de 6 meses
  while pgto_date > start_date:
    pgto_date_list.append(pgto_date)
    pgto_date -= relativedelta(months=6)
  # Inverte a ordem da lista para crescente
  pgto_date_list=pgto_date_list[::-1]
  
  return pgto_date_list

LTN

In [61]:
def calcular_ltn(data_venc, taxa, data_ref):
  duration = dias_uteis(data_ref, data_venc)
  pm = (data_venc - data_ref).days
  pu = 1000/((1+taxa)**(duration/252))
  return pu, duration, pm

NTN-F

In [60]:
def calcular_ntnf(data_venc, taxa, data_ref):
  lista_datas = get_dias_cupom(data_ref, data_venc)
  fluxo_pgto = pd.DataFrame(lista_datas, columns=['data_pgto'])
  fluxo_pgto['cupom_semestral'] = [(1.1**(1/2)-1)*1000]*len(lista_datas)
  fluxo_pgto['cupom_semestral'].iloc[-1] = fluxo_pgto['cupom_semestral'].iloc[-1] + 1000
  fluxo_pgto['dias_uteis'] = fluxo_pgto['data_pgto'].apply(lambda x: dias_uteis(data_ref, x))
  fluxo_pgto['dias_corridos'] = fluxo_pgto['data_pgto'].apply(lambda x: (x - data_ref).days)
  fluxo_pgto['vp_pgto'] = fluxo_pgto.apply(lambda x: (x['cupom_semestral'])/((1+taxa)**(x['dias_uteis']/252)), axis=1)

  pu = fluxo_pgto['vp_pgto'].sum()
  duration = (fluxo_pgto['vp_pgto']*fluxo_pgto['dias_uteis']).sum()/pu
  pm = (fluxo_pgto['cupom_semestral']*fluxo_pgto['dias_corridos']).sum()/fluxo_pgto['cupom_semestral'].sum()

  return pu, duration, pm


NTN-B

In [59]:
def calcular_ntnb(data_venc, taxa, data_ref):
  # d15_base = data_liqui.replace(day=15)
  # d15_proj = d15_base + relativedelta(months=1)
  
  # VNA deveria puxar dados de um banco, porém ainda não existe esse banco e o API tem só uma data em sandbox
  vna = [x['vna'] for x in data_vna[0]['titulos'] if x['codigo_selic']=='760100'][0]

  lista_datas = get_dias_cupom(data_ref, data_venc)
  fluxo_pgto = pd.DataFrame(lista_datas, columns=['data_pgto'])
  fluxo_pgto['fluxo'] = [(1.06**(1/2)-1)]*len(lista_datas)
  fluxo_pgto['fluxo'].iloc[-1] = fluxo_pgto['fluxo'].iloc[-1] + 1
  fluxo_pgto['dias_uteis'] = fluxo_pgto['data_pgto'].apply(lambda x: dias_uteis(data_ref, x))
  fluxo_pgto['dias_corridos'] = fluxo_pgto['data_pgto'].apply(lambda x: (x - data_ref).days)
  fluxo_pgto['vf_fluxo_taxa'] = fluxo_pgto.apply(lambda x: ((1+taxa)**(x['dias_uteis']/252)), axis=1)
  fluxo_pgto['cotacao'] = fluxo_pgto.apply(lambda x: x['fluxo']/x['vf_fluxo_taxa'], axis=1)

  fluxo_pgto['cupom_semestral'] = fluxo_pgto['fluxo']*vna
  fluxo_pgto['vp_pgto'] = fluxo_pgto['cupom_semestral']/fluxo_pgto['vf_fluxo_taxa']

  pu = fluxo_pgto['cotacao'].sum()*vna
  duration = (fluxo_pgto['vp_pgto']*fluxo_pgto['dias_uteis']).sum()/fluxo_pgto['vp_pgto'].sum()
  pm = (fluxo_pgto['cupom_semestral']*fluxo_pgto['dias_corridos']).sum()/fluxo_pgto['cupom_semestral'].sum()
  
  return pu, duration, pm

LFT

In [62]:
def calcular_lft(data_venc, taxa, data_ref):
  duration = dias_uteis(data_ref, data_venc)
  pm = (data_venc - data_ref).days

  vna = [x['vna'] for x in data_vna[0]['titulos'] if x['codigo_selic']=='210100'][0]
  cotacao = (1+taxa)**(-duration/252)
  pu = vna*cotacao
  return pu, duration, pm

## Cálculo Geral

In [ ]:
for titulo in clean_mercado:
  data_ref=datetime.strptime(titulo['data_referencia'], "%Y-%m-%d").date()
  data_venc=datetime.strptime(titulo['data_vencimento'], "%Y-%m-%d").date()

  if titulo['tipo_titulo'] == 'LTN':
    pu_calc = calcular_ltn(
      data_venc=data_venc,
      taxa=titulo['taxa_indicativa']/100,
      data_ref=data_ref,
      )

  elif titulo['tipo_titulo'] == 'NTN-B':
    pu_calc = calcular_ntnb(
      data_venc=data_venc,
      taxa=titulo['taxa_indicativa']/100,
      data_ref=data_ref,
      )

  elif titulo['tipo_titulo'] == 'NTN-F':
    pu_calc = calcular_ntnf(
      data_venc=data_venc,
      taxa=titulo['taxa_indicativa']/100,
      data_ref=data_ref,
      )

  elif titulo['tipo_titulo'] == 'LFT':
    pu_calc = calcular_lft(
      data_venc=data_venc,
      taxa=titulo['taxa_indicativa']/100,
      data_ref=data_ref,
      )

  titulo['pu_calc']=pu_calc

In [ ]:
t =[x for x in clean_mercado if x['tipo_titulo'] in ['NTN-F', 'NTN-B', 'LFT', 'LTN']]
print('Tipo, Data Venc, Real, Calc, Error')
for x in t: print(f"{x['tipo_titulo']}, {x['data_referencia']}, {x['pu']}, {x['pu_calc'][0]}, {check_error(x['pu_calc'][0], x['pu'])}")